In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from google.colab import drive
from tqdm import tqdm
import dill
import numpy as np
import random
import copy
import os
from IPython.display import clear_output
from collections import defaultdict 

In [ ]:
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'rb') as pkl:
    embeddings_dict = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'rb') as pkl:
    ai_word_bank_list = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/topic_dict.pkl', 'rb') as pkl:
    topic_dict = dill.load(pkl)

In [ ]:
#added later for further development
with open('/content/drive/My Drive/Chameleon_AI/Pickles/similarity_dict.pkl', 'rb') as pkl:
    similarity_dict = dill.load(pkl)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"

In [ ]:
USE_model = hub.load(module_url)

In [ ]:
def randomize_secret_word():
    random_topic = random.choice(list(topic_dict))
    topic_word_spread = topic_dict[random_topic]
    secret_word = random.choice(topic_word_spread)
    topic_word_spread.remove(secret_word)
    return random_topic, topic_word_spread, secret_word

In [ ]:
def string_checker(string1, string2):
    string1 = string1.lower()
    string2= string2.lower()
    if len(string1) >= 3:
        if string1[:3] in string2:
            return True
    if len(string2) >= 3:
        if string2[:3] in string1:
            return True
    if len(string1) >= 4 and len(string2) >= 4:
        for index, letter in enumerate(string1):
            if index <= (len(string1) - 4):
                sub_string = string1[index : index + 4]
                if sub_string in string2:
                    return True
    if len(string2) >= 4 and len(string1) >=4:
        for index, letter in enumerate(string2):
            if index <= (len(string2) - 4):
                sub_string = string2[index : index +4]
                if sub_string in string1:
                    return True
    return False

In [ ]:
def check_topic(hint_candidate, topic):
    if similarity_dict[topic][hint_candidate] >= .4 and \
    similarity_dict[topic][hint_candidate] <= .59:
        #print(f"{topic} topic match at{similarity_dict[topic][hint_candidate]}")
        return True
    else:
        #print(f"{topic} failure did not match at " \
              #f"{similarity_dict[topic][hint_candidate]}")
        return False

In [ ]:
def check_non_secret_words(hint_candidate, non_secret_words):
    non_secret_options = copy.deepcopy(non_secret_words)
    non_secret_success_count = 0
    match_list = []
    while len(non_secret_options) >= 1:
        non_secret_word = random.choice(non_secret_options)
        if string_checker(hint_candidate, non_secret_word):
            return False
        if similarity_dict[non_secret_word][hint_candidate] >= .45 and \
        similarity_dict[non_secret_word][hint_candidate] <= .59:
            non_secret_success_count +=1
            match_list.append(non_secret_word)
            non_secret_options.remove(non_secret_word)
        else:
            non_secret_options.remove(non_secret_word)
    
    if non_secret_success_count >= 2:
        #print(f"Success: {hint_candidate} matched with " \
              #f"{non_secret_success_count} words.")
        #print(match_list)
        return True
    else:
        #print(f"Failure: {hint_candidate} matched with " \
              #f"{non_secret_success_count} words.")
        return False

In [ ]:
def hint_search(secret_word, topic, non_secret_words, ai_available_words):
    while len(ai_available_words) >= 1:
        #print(len(ai_available_words))
        hint_candidate = (random.choice(ai_available_words))
        if string_checker(secret_word, hint_candidate):
            ai_available_words.remove(hint_candidate)
            continue
        if similarity_dict[secret_word][hint_candidate] >= .46 and \
            similarity_dict[secret_word][hint_candidate] <= .54:
            #print(f"secret {secret_word} matches with {hint_candidate} at " \
                  #f"{similarity_dict[secret_word][hint_candidate]}.")
            if string_checker(hint_candidate, topic):
                ai_available_words.remove(hint_candidate)
                continue
            if check_topic(hint_candidate, topic):
                if check_non_secret_words(hint_candidate, non_secret_words):
                    return hint_candidate
                else:
                    ai_available_words.remove(hint_candidate)
                    continue            
            else:
                ai_available_words.remove(hint_candidate)
                continue
        else:
            #print(f"Fail: secret {secret_word} no match with {hint_candidate} at " \
                  #f"{similarity_dict[secret_word][hint_candidate]}.")
            ai_available_words.remove(hint_candidate)
            #print('lets try again')
            #print(hint_candidate)
            #print(similarity_dict[secret_word][hint_candidate])

In [ ]:
def chameleon_hint_search(topic, non_secret_words, ai_available_words):
    while len(ai_available_words) >= 1:
        hint_candidate = (random.choice(ai_available_words))
        if string_checker(hint_candidate, topic):
            ai_available_words.remove(hint_candidate)
            continue
        if check_topic(hint_candidate, topic):
            if check_non_secret_words(hint_candidate, non_secret_words):
                return hint_candidate
            else:
                ai_available_words.remove(hint_candidate)
                continue            
        else:
                ai_available_words.remove(hint_candidate)
                continue

In [ ]:
def ai_player_hint(chameleon):
    topic, non_secret_words, secret_word = randomize_secret_word()
    input(f"Topic: {topic}      Secret word: {secret_word}.Confirm to continue.")
    clear_output()
    input(f"Round will now proceed. Think of your hint.")
    ai_available_words = copy.deepcopy(ai_word_bank_list)
    try:
        ai_available_words.remove(topic.lower())
    except ValueError:
        pass

    for word in non_secret_words:
        try:
            ai_available_words.remove(word.lower())
        except ValueError:
            continue

    #print(secret_word)
    still_search = True
    
    if chameleon == 6:
        hint_candidate = chameleon_hint_search(topic, non_secret_words,
                                               ai_available_words)
    else:
        hint_candidate = hint_search(secret_word, topic,
                                 non_secret_words, ai_available_words)
    
    #print(f"hint for {secret_word} is {hint_candidate} at " \
          #f"{similarity_dict[secret_word][hint_candidate]}")


    return secret_word, hint_candidate, topic

In [ ]:
def ai_player_accuse(secret_word, ai_hint, dealer):
    human_hints_list = []
    other_players_similarity_dict = {}
    input('Please confirm when all players are ready to give their hint.')
    clear_output()
    player_1_hint =input("Player 1 what's your hint?")
    human_hints_list.append(player_1_hint)
    player_1_embedding = USE_model([player_1_hint])
    player_2_hint =input("Player 2 what's your hint?")
    human_hints_list.append(player_2_hint)
    player_2_embedding = USE_model([player_2_hint])
    player_3_hint =input("Player 3 what's your hint?")
    human_hints_list.append(player_3_hint)
    player_3_embedding = USE_model([player_3_hint])
    player_4_hint =input("Player 4 what's your hint?")
    human_hints_list.append(player_4_hint)
    player_4_embedding = USE_model([player_4_hint])
    player_5_hint =input("Player 5 what's your hint?")
    human_hints_list.append(player_5_hint)
    player_5_embedding = USE_model([player_5_hint])
    input(f"The AI's hint is the word {ai_hint}. What a remarkable hint!")
    clear_output()
    print("Before we vote, to summarize hints are:")
    print(human_hints_list, ai_hint)
    player_1_similarity = np.inner(player_1_embedding, embeddings_dict[secret_word])
    other_players_similarity_dict['1'] = player_1_similarity
    player_2_similarity = np.inner(player_2_embedding, embeddings_dict[secret_word])
    other_players_similarity_dict['2'] = player_2_similarity
    player_3_similarity = np.inner(player_3_embedding, embeddings_dict[secret_word])
    other_players_similarity_dict['3'] = player_3_similarity
    player_4_similarity = np.inner(player_4_embedding, embeddings_dict[secret_word])
    other_players_similarity_dict['4'] = player_4_similarity
    player_5_similarity = np.inner(player_5_embedding, embeddings_dict[secret_word])
    other_players_similarity_dict['5'] = player_5_similarity
    sort_player_scores = sorted(other_players_similarity_dict.items(),
                                key=lambda x: x[1])
    print(f"I think the most likely Chameleon is {sort_player_scores[0][0]} " \
          f"Who do you think?")
    voted_as_chameleon = voting_time(sort_player_scores, dealer)
    voted_as_chameleon = int(voted_as_chameleon)
    return voted_as_chameleon, human_hints_list

In [ ]:
def voting_time(sort_player_scores, dealer):
    voting_dict = {"1": 0, '2': 0, "3": 0, '4': 0, "5": 0, "6":0}
    tie_list = []
    player_1_vote = input(f"Player 1, who do you vote?")
    voting_dict[player_1_vote] += 1
    player_2_vote = input(f"Player 2, who do you vote?")
    voting_dict[player_2_vote] += 1
    player_3_vote = input(f"Player 3, who do you vote?")
    voting_dict[player_3_vote] += 1
    player_4_vote = input(f"Player 4, who do you vote?")
    voting_dict[player_4_vote] += 1
    player_5_vote = input(f"Player 5, who do you vote?")
    voting_dict[player_5_vote] += 1
    voting_dict[sort_player_scores[0][0]] += 1
    sorted_votes = sorted(voting_dict.items(), key= lambda x: x[1],
                          reverse=True)
    for player_tally in sorted_votes[1:]:
        if player_tally[1] == sorted_votes[0][1]:
            #print(player_tally[1])
            #print(sorted_votes[0][1])
            chameleon_final_vote = input(f"There is a tie. Player {dealer}, who do you choose?")
            return chameleon_final_vote
        else:
            return sorted_votes[0][0]

In [ ]:
def determine_dealer():
    list = [1,2,3,4,5,6]
    first_round_check = input("Is this the first round for this group?")
    if first_round_check == 'y':
        dealer = random.choice(list)
        if dealer == 6:
            input(f"The dealer is the AI. Confirm to continue. ")
            clear_output()
            return dealer
        else:
            input(f"The dealer is Player {dealer}. Confirm to continue. ")
            clear_output()
    else:
        dealer = input('Who was the Chameleon last round?')
        input(f"The dealer is Player {dealer}. Confirm to continue. ")
        clear_output()

    return dealer

In [ ]:
def determine_chameleon():
    list = [1,2,3,4,5,6]
    chameleon = random.choice(list)
    input("Choosing Chameleon, all players turn away please.")
    input(f"The Chameleon is Player {chameleon}. How exciting! Confirm to continue.")
    clear_output()
    return chameleon

In [ ]:
def chameleon_secret_guess(human_hints_list, topic, topic_dict,
                           embeddings_dict):
    top_words = defaultdict(int)
    for hint in human_hints_list:
        highest_score = 0
        hint_vector = USE_model([hint])
        for secret_word in topic_dict[topic]:
            score = np.inner(hint_vector, embeddings_dict[secret_word])
            if score > highest_score:
                highest_score = score
                most_likely_secret = secret_word
        top_words[most_likely_secret] +=1
    secret_guess = (max(top_words, key=top_words.get))
    return secret_guess            

In [ ]:
chameleon_secret_guess(["wheels","drive"," gas", "ride", "speed"], "Transport", topic_dict, embeddings_dict)

In [ ]:
def demo_run():
    dealer = determine_dealer()
    chameleon = determine_chameleon()
    secret_word, ai_hint, topic = ai_player_hint(chameleon)
    voted_as_chameleon, human_hints_list = ai_player_accuse(secret_word,
                                                            ai_hint, dealer)
    print(f"{voted_as_chameleon} has been voted as Chameleon.")
    input(f"Drumroll please... Chameleon, reveal yourself!")
    clear_output()
    if chameleon != voted_as_chameleon:
       print(f"The Chameleon escaped detection! It was actually player {chameleon}.")
       print("The Chameleon gets 2 points, everyone else gets 0.")
    else:
        input(f"OH SNAP, the Chameleon was discovered!")
        clear_output()
        if chameleon == 6:
            secret_guess = chameleon_secret_guess(human_hints_list, topic,
                                                  topic_dict, embeddings_dict)
            if secret_guess == secret_word:
                print(f"The AI choose correctly with {secret_guess}.")
                input(f"Chameleon gets 1 point, everyone else gets 0.")
            else:
                print(f"The AI chose incorrectly with {secret_guess}. The secret word was actually {secret_word}.")
                input(f"The Chameleon gets 0 points, all other players get 2 points.")
        else:
            secret_guess = input(f"Player {chameleon}, what is your guess?")
            if secret_guess == secret_word:
                print(f"Player {chameleon} chose correctly!")
                print("Chameleon gets 1 point, everyone else gets 0.")
            else:
                print(f"Player {chameleon} chose incorrectly. The secret word was actually {secret}.")
                input(f"The Chameleon gets 0 points, all ohter players get points.")
                
    play_again = input("Play again?")
    clear_output()
    if play_again == 'y':
        demo_run()
    else:
        return "Thanks for playing along."

In [ ]:
demo_run()